In [303]:
"""
Скрипт для обработки форм мониторинга кадровой потребности ОПК августовская форма
"""

'\nСкрипт для обработки форм мониторинга кадровой потребности ОПК августовская форма\n'

In [304]:
# -*- coding: UTF-8 -*-
import pandas as pd
import numpy as np
import tkinter
import sys
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
# pd.options.mode.chained_assignment = None  # default='warn'
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
import copy
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import random
pd.options.display.width = 0

In [305]:
path_data_opk = 'data/Форма август'
path_to_end_folder = 'data'
file_form_data = 'data/Форма сбора данных.xlsx'

In [306]:
# дождусь пока форма не устаканится и потом добавллю эту проверку
lst_mon_col = [f'гр.{i}' for i in range(1,26)] # создаем список для проверки колонок данных по мониторингу
lst_act_col = [f'гр.{i}' for i in range(1,13)] # создаем список для проверки колонок данных по мерам


In [307]:
lst_act_col

['гр.1',
 'гр.2',
 'гр.3',
 'гр.4',
 'гр.5',
 'гр.6',
 'гр.7',
 'гр.8',
 'гр.9',
 'гр.10',
 'гр.11',
 'гр.12']

In [308]:
# Создаем итоговые датафреймы
base_mon_df = pd.DataFrame(columns=lst_mon_col)
base_action_df = pd.DataFrame(columns=lst_act_col)

In [309]:
error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ]) # датафрейм с ошибками


In [310]:
for file in os.listdir(path_data_opk):
    if not file.startswith('~$') and file.endswith('.xls'):
        name_file = file.split('.xls')[0]
        temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                            'Файл с расширением XLS (СТАРЫЙ ФОРМАТ EXCEL)!!! ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                     columns=['Название файла', 'Строка или колонка с ошибкой',
                                              'Описание ошибки'])
        error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
        continue
    if not file.startswith('~$') and file.endswith('.xlsx'):
        name_file = file.split('.xlsx')[0]
        print(name_file)
        # Проверяем наличие листов с названиями Форма 1 и Форма 2
        wb_1 = openpyxl.load_workbook(f'{path_data_opk}/{file}')
        if not {'Форма по мониторингу','Форма по принимаемым мерам'}.issubset(set(wb_1.sheetnames)):
            temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                'Проверьте наличие листов с названием Форма по мониторингу и Форма по принимаемым мерам! Не должно быть пробелов в начале и в конце названия ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                         columns=['Название файла', 'Строка или колонка с ошибкой',
                                                  'Описание ошибки'])
            error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
            continue
            
    
    # считываем датафреймы
    df_mon = pd.read_excel(f'{path_data_opk}/{file}',sheet_name='Форма по мониторингу',dtype=str,skiprows=3,header=None) # лист с данными мониторинга
    df_action = pd.read_excel(f'{path_data_opk}/{file}',sheet_name='Форма по принимаемым мерам',dtype=str,skiprows=3,header=None) # лист с данными по принимаемым мерам
    
    # Присваиваем названия колонкам
    df_mon.columns = lst_mon_col
    df_action.columns = lst_act_col

    
    # удаляем полностью пустые строки
    df_mon.dropna(thresh=7,inplace=True)
    df_action.dropna(thresh=7,inplace=True)
    
    # Проверяем совпадение ИНН.
    """
    Проверка на ошибки
    """
    
    # добавляем название файла
    df_mon['Название файла'] =name_file
    df_action['Название файла'] = name_file
    
    # добавляем в базовые датафреймы
    base_mon_df = pd.concat([base_mon_df,df_mon],ignore_index=True)
    base_action_df = pd.concat([base_action_df,df_action],ignore_index=True)
    
    
    
        

Неправильное название
Отсутствует лист
Тестовая №1
Тестовая №2


In [311]:
df_mon

,гр.1,гр.2,гр.3,гр.4,гр.5,гр.6,гр.7,гр.8,гр.9,гр.10,...,гр.17,гр.18,гр.19,гр.20,гр.21,гр.22,гр.23,гр.24,гр.25,Название файла
0,Республика Бурятия,0323053578,"АО ""У-У ППО""",Потребность имеется,Профессии рабочих,Авербандщик,2,29.02.08 Технология обработки алмазов,10,6,...,1,1,1,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО,Тестовая №2
1,Республика Бурятия,0323053555,ООО Тест,Потребность отсутствует,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО,Тестовая №2


In [312]:
df_action

,гр.1,гр.2,гр.3,гр.4,гр.5,гр.6,гр.7,гр.8,гр.9,гр.10,гр.11,гр.12,Название файла


In [313]:
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
with pd.ExcelWriter(f'{path_to_end_folder}/Сводный файл ОПК от {current_time}.xlsx') as writer:
    base_mon_df.to_excel(writer,sheet_name='Форма по мониторингу',index=False)
    base_action_df.to_excel(writer,sheet_name='Форма по принимаемым мерам',index=False)

In [314]:
error_df

,Название файла,Строка или колонка с ошибкой,Описание ошибки
0,2003,,Файл с расширением XLS (СТАРЫЙ ФОРМАТ EXCEL)!!...
1,Неправильное название,,Проверьте наличие листов с названием Форма по ...
2,Отсутствует лист,,Проверьте наличие листов с названием Форма по ...
